In [38]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import os
from dotenv import load_dotenv
import pyfredapi as pf
import requests
import re

In [3]:
load_dotenv()
# get my FRED_API_KEY from my local environment
api_key = os.environ['FRED_API_KEY']

In [113]:
"""
Business confidence index (BCI) - https://data.oecd.org/leadind/business-confidence-index-bci.htm - 1974
Consumer confidence index (CCI) - https://data.oecd.org/leadind/consumer-confidence-index-cci.htm#indicator-chart - 1974
Composite leading indicator (CLI) - https://data.oecd.org/leadind/composite-leading-indicator-cli.htm#indicator-chart - 1974
"""

'\nBusiness confidence index (BCI) - https://data.oecd.org/leadind/business-confidence-index-bci.htm - 1974\nConsumer confidence index (CCI) - https://data.oecd.org/leadind/consumer-confidence-index-cci.htm#indicator-chart - 1974\nComposite leading indicator (CLI) - https://data.oecd.org/leadind/composite-leading-indicator-cli.htm#indicator-chart - 1974\n'

In [4]:
data_sources = """
(FEDFUNDS)
Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUCSL)
Sticky Price Consumer Price Index less Food and Energy (CORESTICKM159SFRBATL)
Sticky Price Consumer Price Index less Food, Energy, and Shelter (CRESTKCPIXSLTRM159SFRBATL)
Producer Price Index by Commodity: All Commodities (PPIACO)
Personal Consumption Expenditures (PCE) 
Total Nonfarm Private Payroll Employment (ADPWNUSNERSA)
Quarterly Financial Report: U.S. Corporations: All Information: Total Cash on Hand and in U.S. Banks  (QFRTCASHINFUSNO) - only to 2009 
Unemployment Rate (UNRATE)
Noncyclical Rate of Unemployment (NROU)
Unemployment Rate - Women (LNS14000002)
Job Openings: Total Nonfarm (JTSJOL) - begins 2000
Layoffs and Discharges: Total Nonfarm (JTSLDL) - since 2000 Gross Domestic Product (GDP)
Real Gross Domestic Product (GDPC1)
Real gross domestic product per capita (A939RX0Q048SBEA)
Gross Domestic Product: Implicit Price Deflator (A191RI1Q225SBEA)
National Accounts: National Accounts Deflators: Gross Domestic Product: GDP Deflator for United States (USAGDPDEFQISMEI)
Advance Retail Sales: Retail Trade and Food Services (RSAFS) - 1992 
University of Michigan: Consumer Sentiment (UMCSENT)
New Privately-Owned Housing Units Started: Total Units (HOUST)
New Privately-Owned Housing Units Started: Single-Family Units (HOUST1F)
Total Business Sales (TOTBUSSMSA) -1992

Nonfinancial Corporate Business; Inventories Including IVA, Market Value Levels (BOGZ1LM105020005Q)
Corporate Profits After Tax -without IVA and CCAdj- (CP)
 National income: Corporate profits before tax -without IVA and CCAdj- (A053RC1Q027SBEA)
Money Market Funds; Total Financial Assets, Level (MMMFFAQ027S)
Stock Market Capitalization to GDP for United States (DDDM01USA156NWDB)
Interest Rates: Long-Term Government Bond Yields: 10-Year: Main -Including Benchmark- for United States (IRLTLT01USM156N)
Nominal Broad U.S. Dollar Index (DTWEXBGS) - 2006
Leading Indicators OECD: Leading indicators: CLI: Amplitude adjusted for OECD - Total (OECDLOLITOAASTSAM)
Trade Balance: Goods and Services, Balance of Payments Basis (BOPGSTB) - 1992
M1 (WM1NS)
Velocity of M1 Money Stock (M1V)
M2 (WM2NS)
"""

In [5]:
# Regular expression pattern to match text within parentheses
pattern = r"\((.*?)\)"

# Find all matches and store them in a list
matches = re.findall(pattern, data_sources)

In [6]:
dataframes = {}

In [32]:
start = '1990-01-01'
# end with the current date
end = pd.Timestamp.today().strftime('%Y-%m-%d')

extra_parameters = {
    "observation_start": start,
    "observation_end": end,
}
for match in matches:
    series_id = match
    try:
        df = pf.get_series(series_id=series_id, **extra_parameters)[['date', 'value']]
        # add the match to the end of the value column name
        df.rename(columns={'value': f'value_{match}'}, inplace=True)
        # turn the date column into a datetime object
        df['date'] = pd.to_datetime(df['date'])

        # set the date column as the index
        df.set_index('date', inplace=True)
        dataframes[series_id] = df

    except:
        print(f"Failed to get {series_id}")
# gdp_df = pf.get_series(series_id="FEDFUNDS", **extra_parameters)
# display(gdp_df)

In [35]:
def align_dataframes(dataframes):
    aligned_dfs = []
    start_date = '1990-01-01'

    for df in dataframes:
        # Ensure the index is a DateTimeIndex
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        # Resample to monthly frequency
        # Use 'mean' for downsampling and 'ffill' for upsampling
        resampled_df = df.resample('M').mean().ffill()

        # Align start date to January 1990, filling missing values with NaN
        aligned_df = resampled_df.reindex(pd.date_range(start_date, resampled_df.index.max(), freq='M'), fill_value=np.nan)
        
        aligned_dfs.append(aligned_df)

    return aligned_dfs

In [37]:
dataframes_list = list(dataframes.values())

# Align all DataFrames to the same frequency and start date
aligned_dataframes = align_dataframes(dataframes_list)

# Concatenate all DataFrames along the column axis
combined_df = pd.concat(aligned_dataframes, axis=1)

# Display the combined DataFrame
print(combined_df)


            value_FEDFUNDS  value_CPIAUCSL  value_CORESTICKM159SFRBATL  \
1990-01-31            8.23         127.500                    4.920589   
1990-02-28            8.24         128.000                    4.935526   
1990-03-31            8.28         128.600                    5.208701   
1990-04-30            8.26         128.900                    5.294134   
1990-05-31            8.18         129.100                    5.183754   
...                    ...             ...                         ...   
2023-07-31            5.12         304.348                    5.456089   
2023-08-31            5.33         306.269                    5.217343   
2023-09-30            5.33         307.481                    5.008782   
2023-10-31            5.33         307.619                    4.931956   
2023-11-30            5.33             NaN                         NaN   

            value_CRESTKCPIXSLTRM159SFRBATL  value_PPIACO  value_PCE  \
1990-01-31                         4.82

In [ ]:
# # get tags
# request_url = f'https://api.stlouisfed.org/fred/tags'

# params = {
#     'api_key': api_key,
#     'file_type': 'json',
#     'limit': 1000,
#     'order_by': 'popularity',
#     # 'offset': 0,
# }
# tags_response = requests.get(request_url, params=params)
# # response_json = tags_response.json()
# # tags_df = pd.DataFrame(response_json['tags'])
# # display(tags_df)
# # print(tags_df['name'].to_list())

In [106]:
# response_json = tags_response.json()
# tags_df = pd.DataFrame(response_json['tags'])
# display(tags_df)

In [107]:
# request_url = f'https://api.stlouisfed.org/fred/releases'

# params = {
#     'api_key': api_key,
#     'file_type': 'json',
#     'limit': 1000,
#     # 'offset': 10,
# }
# response = requests.get(request_url, params=params)
# # # print the 'name' for each Release
# # for release in response.json()['releases']:
# #     print(release['name'])

In [109]:
# # Define the path to the Vol 3 Data
# folder_path = 'VOL3-DATA'

# # Dictionary to hold DataFrames, keyed by file name
# dataframes = {}

# # Iterate over each file in the folder
# for filename in os.listdir(folder_path):
#     # Check if the file is a CSV
#     if filename.endswith('.csv'):
#         file_path = os.path.join(folder_path, filename)
#         try:
#             # Read the CSV file into a DataFrame, skip bad lines
#             dataframes[filename] = pd.read_csv(file_path, on_bad_lines='skip')
#         except Exception as e:
#             print(f"Error reading {filename}: {e}")
